In [ ]:
#### Sweep processing code ####
# This file contains the code handling the measurements from the payload sweep experiments.

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from scipy import signal as sig
from scipy import stats as stat
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from numpy import fft
import importlib

import processing_helpers as ph
from processing_helpers import do_linear_regression
from processing_helpers import get_residuals
from processing_helpers import segment_data
from processing_helpers import plot_pred_interval

In [ ]:
## For reloading processing_helpers when changes are made
importlib.reload(ph)

In [ ]:
#initial constants
delta_t = 0.00025 #s
V = 3.7 #V

to_uWh = (1000*1000*delta_t*V)/3600 #uWh

nb_active_energy = 28.48 #uWh Based on 20 measurements from test data
nb_active_time = 20.48 #s

ltem_active_energy = 101.95 #uWh Based on 20 measurements from test data
ltem_active_time = 10.24 #s

In [ ]:
## CoAP measurment data
coap_byte_step = 41

sweep_file1 = "../measurements/otii/sweep_coap_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_coap_psm_test02.csv" 
sweep_file3 = "../measurements/otii/sweep_coap_psm_test03.csv" 


coap_nb_data = []
coap_nb_data.append(pd.read_csv(sweep_file1))
coap_nb_data.append(pd.read_csv(sweep_file2))
coap_nb_data.append(pd.read_csv(sweep_file3))


sweep_file1 = "../measurements/otii/sweep_ltem_coap_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_ltem_coap_psm_test02.csv" 
sweep_file3 = "../measurements/otii/sweep_ltem_coap_psm_test03.csv"


coap_ltem_data = []
coap_ltem_data.append(pd.read_csv(sweep_file1))
coap_ltem_data.append(pd.read_csv(sweep_file2))
coap_ltem_data.append(pd.read_csv(sweep_file3))


In [ ]:
## MQTT measurement data
mqtt_byte_step = 64

sweep_file1 = "../measurements/otii/sweep_mqtt_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_mqtt_psm_test02.csv" 
sweep_file3 = "../measurements/otii/sweep_mqtt_psm_test03.csv" 


mqtt_data = []
mqtt_data.append(pd.read_csv(sweep_file1))
mqtt_data.append(pd.read_csv(sweep_file2))
mqtt_data.append(pd.read_csv(sweep_file3))

sweep_file1 = "../measurements/otii/sweep_ltem_mqtt_psm_test01.csv"
sweep_file2 = "../measurements/otii/sweep_ltem_mqtt_psm_test02.csv"
sweep_file3 = "../measurements/otii/sweep_ltem_mqtt_psm_test03.csv"

mqtt_ltem_data = []
mqtt_ltem_data.append(pd.read_csv(sweep_file1))
mqtt_ltem_data.append(pd.read_csv(sweep_file2))
#mqtt_ltem_data.append(pd.read_csv(sweep_file3)) #major outlier that affects segmenting



In [ ]:
## Getting the Energy and Current data from the measurements

coap_nb_energy = []
coap_nb_current = []
for i in coap_nb_data:
    coap_nb_energy.append(i["Arc Main Energy (J)"])
    coap_nb_current.append(i["Arc Main Current (A)"])

mqtt_nb_energy = []
mqtt_nb_current = []
for i in mqtt_data:
    mqtt_nb_energy.append(i["Arc Main Energy (J)"])
    mqtt_nb_current.append(i["Arc Main Current (A)"])

coap_ltem_energy = []
coap_ltem_current = []
for i in coap_ltem_data:
    coap_ltem_energy.append(i["Arc Main Energy (J)"])
    coap_ltem_current.append(i["Arc Main Current (A)"])

mqtt_ltem_energy = []
mqtt_ltem_current = []
for i in mqtt_ltem_data:
    mqtt_ltem_energy.append(i["Arc Main Energy (J)"])
    mqtt_ltem_current.append(i["Arc Main Current (A)"])

In [ ]:
## Plotting total energy

plt.figure()
for i in coap_nb_energy:
    i.plot()

for i in coap_ltem_energy:
    i.plot()

for i in mqtt_nb_energy:
    i.plot()

for i in mqtt_ltem_energy:
    i.plot()   

In [ ]:
#### CoAP over NB-IoT - START   ####

In [ ]:
## Segmenting the CoAP over NB-IoT measurements
[segments_coap_nb, timing_coap_nb] = segment_data(coap_nb_current, 160000, 0.01, 120000, 20000)

In [ ]:
## Calculating the energy and time spent in each segment of each data set

fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0
coap_nb_sums = []

bytes_coap = range(0,coap_byte_step*len(segments_coap_nb[0]), coap_byte_step)

for i in segments_coap_nb:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh - nb_active_energy
        sum_acc.append(curr_sum)
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_coap,sum_acc, label = str(idx_seg))
    coap_nb_sums.append(sum_acc)
    idx_seg += 1

    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
## Calulating the average time and energy spent for each payload step
coap_nb_stds = []
coap_nb_avg = []

timing_coap_nb_avg = []

for i in range(len(coap_nb_sums[0])):
    curr_sample = []
    curr_time = []
    for j in range(len(coap_nb_sums)):
        curr_sample.append(coap_nb_sums[j][i])
        curr_time.append(timing_coap_nb[j][i])
    coap_nb_avg.append(np.mean(curr_sample))
    coap_nb_stds.append(np.std(curr_sample))
   
    timing_coap_nb_avg.append(np.mean(curr_time))

print(np.mean(coap_nb_stds))
print(coap_nb_avg[1])

In [ ]:
#### CoAP over NB-IoT - END   ####


#### CoAP over LTE-M  - START ####

In [ ]:
## Segmenting the CoAP over LTE-M measurements
[segments_coap_ltem, timing_coap_ltem] = segment_data(coap_ltem_current, 100000, 0.01, 60000, 20000)

In [ ]:
## Calculating the energy and time spent in each segment of each data set
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

coap_ltem_sums = []

for i in segments_coap_ltem:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh - ltem_active_energy
        sum_acc.append(curr_sum)
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_coap,sum_acc, label = str(idx_seg))
    coap_ltem_sums.append(sum_acc)
    idx_seg += 1
    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
## Calulating the average time and energy spent for each payload step
coap_ltem_stds = []
coap_ltem_avg = []

timing_coap_ltem_avg = []

for i in range(len(coap_ltem_sums[0])):
    curr_sample = []
    curr_time = []
    for j in range(len(coap_ltem_sums)):
        curr_sample.append(coap_ltem_sums[j][i])
        curr_time.append(timing_coap_ltem[j][i])
    coap_ltem_avg.append(np.mean(curr_sample))
    coap_ltem_stds.append(np.std(curr_sample))

    timing_coap_ltem_avg.append(np.mean(curr_time))

   
print(np.mean(coap_ltem_stds))
print(coap_ltem_avg[1])

In [ ]:
#### COAP over LTE-M  -  END  ####

#### MQTT over NB-IoT - START ####

In [ ]:
## Segmenting the MQTT over NB-IoT measurements
[segments_mqtt_nb, timing_mqtt_nb] = segment_data(mqtt_nb_current, 180000, 0.01, 140000, 20000)

In [ ]:
## Calculating the energy and time spent in each segment of each data set
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

mqtt_nb_sums = []
bytes_mqtt = range(0,mqtt_byte_step*len(segments_mqtt_nb[0]), mqtt_byte_step)

for i in segments_mqtt_nb:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh - nb_active_energy
        sum_acc.append(curr_sum)
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_mqtt,sum_acc, label = str(idx_seg))
    mqtt_nb_sums.append(sum_acc)
    idx_seg += 1
    
plt.grid()
    
plt.ylabel("Power consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
## Calulating the average time and energy spent for each payload step
mqtt_nb_stds = []
mqtt_nb_avg = []

timing_mqtt_nb_avg = []

for i in range(len(mqtt_nb_sums[0])):
    curr_sample = []
    curr_time = []
    for j in range(len(mqtt_nb_sums)):
        curr_sample.append(mqtt_nb_sums[j][i])
        curr_time.append(timing_mqtt_nb[j][i])
    mqtt_nb_avg.append(np.mean(curr_sample))
    mqtt_nb_stds.append(np.std(curr_sample))

    timing_mqtt_nb_avg.append(np.mean(curr_time))

   
print(np.mean(mqtt_nb_stds))
print(mqtt_nb_avg[1])
print(len(mqtt_nb_avg))

In [ ]:
#### MQTT over NB-IoT -  END  ####

#### MQTT over LTE-M  - START ####

In [ ]:
## Segmenting the MQTT over LTE-M measurements
[segments_mqtt_ltem, timing_mqtt_ltem] = segment_data(mqtt_ltem_current, 120000, 0.01, 100000, 20000)


In [ ]:
## Calculating the energy and time spent in each segment of each data set
fig = plt.figure()
ax = fig.add_subplot(111)

idx_seg = 0

mqtt_ltem_sums = []
bytes_mqtt = range(0,mqtt_byte_step*len(segments_mqtt_ltem[0]), mqtt_byte_step)

for i in segments_mqtt_ltem:
    sum_acc = []
    idx_sample = 0
    for j in i:
        curr_sum = j.sum()*to_uWh - ltem_active_energy
        sum_acc.append(curr_sum)
        #mqtt_ltem_sums[idx_sample][idx_seg] = curr_sum
        idx_sample += 1
    
    print(np.size(sum_acc))
    plt.plot(bytes_mqtt,sum_acc, label = str(idx_seg))
    mqtt_ltem_sums.append(sum_acc)
    idx_seg += 1
    
plt.grid()
    
plt.ylabel("Energy consumption [uWh]")
plt.xlabel("Payload size [Bytes]")

In [ ]:
## Calulating the average time and energy spent for each payload step

mqtt_ltem_stds = []
mqtt_ltem_avg = []

timing_mqtt_ltem_avg = []

for i in range(len(mqtt_ltem_sums[0])):
    curr_sample = []
    curr_time = []
    for j in range(len(mqtt_ltem_sums)):
        curr_sample.append(mqtt_ltem_sums[j][i])
        curr_time.append(timing_mqtt_ltem[j][i])

    mqtt_ltem_avg.append(np.mean(curr_sample))
    mqtt_ltem_stds.append(np.std(curr_sample))
    
    timing_mqtt_ltem_avg.append(np.mean(curr_time))

print(np.mean(mqtt_ltem_stds))
print(mqtt_ltem_avg[1])
print(len(mqtt_ltem_avg))

In [ ]:
## Smoothing the average measurements for pretty plotting
#  Applying a Savitzky-Golay filter:
#  - window length   : 9
#  - polynomial order: 3

coap_ltem_smooth = sig.savgol_filter(coap_ltem_avg,9,3)
coap_nb_smooth   = sig.savgol_filter(coap_nb_avg,9,3)
mqtt_nb_smooth   = sig.savgol_filter(mqtt_nb_avg,9,3)
mqtt_ltem_smooth = sig.savgol_filter(mqtt_ltem_avg,9,3)

timing_coap_ltem_smooth = sig.savgol_filter(timing_coap_ltem_avg,9,3)
timing_coap_nb_smooth   = sig.savgol_filter(timing_coap_nb_avg,9,3)
timing_mqtt_nb_smooth   = sig.savgol_filter(timing_mqtt_nb_avg,9,3)
timing_mqtt_ltem_smooth = sig.savgol_filter(timing_mqtt_ltem_avg,9,3)

In [ ]:
## Plotting the smoothed energy averages
fig, (ax1, ax2) = plt.subplots(1,2)

plt.suptitle("Message energy comparison")


ax1.set_title('NB-IoT')
ax2.set_title('LTE-M')

ax1.set(ylabel="Energy [uWh]")
ax1.set_xlabel("Payload size [Bytes]")
ax2.set_xlabel("Payload size [Bytes]")
ax1.set_ylim([20,200])
ax2.set_ylim([20,200])

ax1.axvline(536)
ax2.axvline(536, label = "536 Bytes (TCP default MSS)")


#plt.plot(bytes_coap, coap_nb_avg   ,     label = "CoAP NB-IoT average")
ax1.plot(bytes_coap, coap_nb_smooth, "g" ,label = "NB-IoT")
ax1.plot(bytes_coap, coap_nb_smooth+nb_active_energy, "g--" ,label = "NB-IoT")


#plt.plot(bytes_mqtt, mqtt_nb_avg   ,       label = "MQTT 2300 average")
ax1.plot(bytes_mqtt, mqtt_nb_smooth, "r" ,label = "NB-IoT")
ax1.plot(bytes_mqtt, mqtt_nb_smooth+nb_active_energy, "r--" ,label = "NB-IoT")


#plt.plot(bytes_coap, coap_ltem_avg   ,     label = "CoAP LTE-M average")
ax2.plot(bytes_coap, coap_ltem_smooth, "g", label = "CoAP")
ax2.plot(bytes_coap, coap_ltem_smooth+ltem_active_energy, "g--", label = "CoAP with cDRX energy")

#plt.plot(bytes_mqtt, mqtt_ltem_avg   ,       label = "MQTT 2300 average")
ax2.plot(bytes_mqtt, mqtt_ltem_smooth, "r" ,label = "MQTT")
ax2.plot(bytes_mqtt, mqtt_ltem_smooth+ltem_active_energy, "r--" ,label = "MQTT with cDRX energy")

ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig("../dt_thesis/plots/tx_energy_comparison.pdf", bbox_inches='tight')

In [ ]:
## Plotting the smoothed time averages
fig, (ax1, ax2) = plt.subplots(1,2)

ax1.set_title('NB-IoT')
ax2.set_title('LTE-M')

plt.suptitle("Transmission time comparison")

ax1.set(ylabel="Transmission time [s]")
ax1.set_xlabel("Payload size [Bytes]")
ax2.set_xlabel("Payload size [Bytes]")
ax1.set_ylim([0,35])
ax2.set_ylim([0,35])

#plt.plot(bytes_coap, coap_nb_avg   ,     label = "CoAP NB-IoT average")
ax1.plot(bytes_coap, timing_coap_nb_smooth-20.48, "g" ,label = "NB-IoT")
ax1.plot(bytes_coap, timing_coap_nb_smooth, "g--" ,label = "NB-IoT")


#plt.plot(bytes_mqtt, mqtt_nb_avg   ,       label = "MQTT 2300 average")
ax1.plot(bytes_mqtt, timing_mqtt_nb_smooth-20.48, "r" ,label = "NB-IoT")
ax1.plot(bytes_mqtt, timing_mqtt_nb_smooth, "r--" ,label = "NB-IoT")


#plt.plot(bytes_coap, coap_ltem_avg   ,     label = "CoAP LTE-M average")
ax2.plot(bytes_coap, timing_coap_ltem_smooth-10.24, "g", label = "CoAP")
ax2.plot(bytes_coap, timing_coap_ltem_smooth, "g--", label = "CoAP with RRC inactive time")


#plt.plot(bytes_mqtt, mqtt_ltem_avg   ,       label = "MQTT 2300 average")
ax2.plot(bytes_mqtt, timing_mqtt_ltem_smooth-10.24, "r" ,label = "MQTT")
ax2.plot(bytes_mqtt, timing_mqtt_ltem_smooth, "r--" ,label = "MQTT with RRC inactive time")

ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig("../dt_thesis/plots/tx_time_comparison.pdf", bbox_inches='tight')

In [ ]:
## OLS linear regression analysis of the calculated message energy 
class Results:
    def __init__(self, results):
        self.intercept = results.params[0]
        self.slope     = results.params[1]
        self.results   = results


timing_coap_nb_res = Results(do_linear_regression(bytes_coap, timing_coap_nb))
timing_mqtt_nb_res = Results(do_linear_regression(bytes_mqtt, timing_mqtt_nb))
timing_coap_ltem_res = Results(do_linear_regression(bytes_coap, timing_coap_ltem))
timing_mqtt_ltem_res = Results(do_linear_regression(bytes_mqtt, timing_mqtt_ltem))
energy_coap_nb_res = Results(do_linear_regression(bytes_coap, coap_nb_sums))
energy_mqtt_nb_res = Results(do_linear_regression(bytes_mqtt, mqtt_nb_sums))
energy_coap_ltem_res = Results(do_linear_regression(bytes_coap, coap_ltem_sums))
energy_mqtt_ltem_res = Results(do_linear_regression(bytes_mqtt, mqtt_ltem_sums))




In [ ]:
## Writing the results to file in order for later use (in the model)

result_matrix = {
    'timing_coap_nb'   : [  timing_coap_nb_res.slope  , timing_coap_nb_res.intercept],
    'energy_coap_nb'   : [  energy_coap_nb_res.slope  , energy_coap_nb_res.intercept],
    'timing_coap_ltem' : [timing_coap_ltem_res.slope, timing_coap_ltem_res.intercept],
    'energy_coap_ltem' : [energy_coap_ltem_res.slope, energy_coap_ltem_res.intercept],
    'timing_mqtt_nb'   : [  timing_mqtt_nb_res.slope  , timing_mqtt_nb_res.intercept],
    'energy_mqtt_nb'   : [  energy_mqtt_nb_res.slope  , energy_mqtt_nb_res.intercept],
    'timing_mqtt_ltem' : [timing_mqtt_ltem_res.slope, timing_mqtt_ltem_res.intercept],
    'energy_mqtt_ltem' : [energy_mqtt_ltem_res.slope, energy_mqtt_ltem_res.intercept]
}

regression_results = pd.DataFrame(data = result_matrix)
regression_results.to_csv("reg_res.csv")

In [ ]:
## Regression line plot for both NB-IoT and LTE-M data.

fig, (ax1,ax2) = plt.subplots(1,2)

fig.suptitle("Message energy regression lines")
x_mqtt = np.linspace(0,4096,4096)
x_coap = np.linspace(0,1439,1439)

## CoAP
ax1.set_title("CoAP")
ax1.set_ylabel("Energy [uWh]")
ax1.set_xlabel("Payload size [Bytes]")
ax1.set_ylim([0,170])

y_coap_nb =   energy_coap_nb_res.slope*x_coap+energy_coap_nb_res.intercept
y_coap_ltem = energy_coap_ltem_res.slope*x_coap+energy_coap_ltem_res.intercept

ax1.plot(x_coap,y_coap_nb,    "g", label="NB-IoT regression line")
ax1.scatter(bytes_coap, coap_nb_sums[0], color = "g", s=5, alpha=0.3, label="NB-IoT data")
ax1.scatter(bytes_coap, coap_nb_sums[1], color = "g", s=5, alpha=0.3)   
ax1.scatter(bytes_coap, coap_nb_sums[2], color = "g", s=5, alpha=0.3)   

ax1.plot(x_coap,y_coap_ltem,  "r", label="LTE-M regression line")
ax1.scatter(bytes_coap, coap_ltem_sums[0], color = "r", s=5, alpha=0.3, label="LTE-M data")
ax1.scatter(bytes_coap, coap_ltem_sums[1], color = "r", s=5, alpha=0.3)   
ax1.scatter(bytes_coap, coap_ltem_sums[2], color = "r", s=5, alpha=0.3)   

## MQTT
ax2.set_title("MQTT")
ax2.set_xlabel("Payload size [Bytes]")
ax2.set_ylim([0,170])


y_mqtt_nb   = energy_mqtt_nb_res.slope  * x_mqtt+energy_mqtt_nb_res.intercept
y_mqtt_ltem = energy_mqtt_ltem_res.slope* x_mqtt+energy_mqtt_ltem_res.intercept

ax2.plot(x_mqtt,y_mqtt_nb,    "g", label="NB-IoT regression line")
ax2.scatter(bytes_mqtt, mqtt_nb_sums[0], color = "g", s=5, alpha=0.3, label="NB-IoT data")
ax2.scatter(bytes_mqtt, mqtt_nb_sums[1], color = "g", s=5, alpha=0.3)   
ax2.scatter(bytes_mqtt, mqtt_nb_sums[2], color = "g", s=5, alpha=0.3)   

ax2.plot(x_mqtt,y_mqtt_ltem,  "r", label="LTE-M regression line")
ax2.scatter(bytes_mqtt, mqtt_ltem_sums[0], color = "r", s=5, alpha=0.3, label="LTE-M data")
ax2.scatter(bytes_mqtt, mqtt_ltem_sums[1], color = "r", s=5, alpha=0.3)   
#ax2.scatter(bytes_mqtt, mqtt_ltem_sums[2], color = "r", s=5, alpha=0.3)   



ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.savefig("../dt_thesis/plots/energy_reg_lines.pdf", bbox_inches='tight')

intersect_coap = (energy_coap_nb_res.intercept-energy_coap_ltem_res.intercept)/(energy_coap_ltem_res.slope-energy_coap_nb_res.slope)
intersect_mqtt = (energy_mqtt_nb_res.intercept-energy_mqtt_ltem_res.intercept)/(energy_mqtt_ltem_res.slope-energy_mqtt_nb_res.slope)


print("CoAP Energy consumption intersection at: " + str(intersect_coap))
print("MQTT Energy consumption intersection at: " + str(intersect_mqtt))



In [ ]:
## Regression line plot for both NB-IoT and LTE-M data.
fig, (ax1,ax2) = plt.subplots(1,2)

fig.suptitle("Transmission time regression lines")
x_mqtt = np.linspace(0,4096,4096)
x_coap = np.linspace(0,1439,1439)

y_min_t = 0
y_max_t = 14

ax1.set_ylim([y_min_t,y_max_t])
ax2.set_ylim([y_min_t,y_max_t])


## CoAP
ax1.set_title("CoAP")
ax1.set_ylabel("Time [s]")
ax1.set_xlabel("Payload size [Bytes]")

y_coap_nb_t =   timing_coap_nb_res.slope  *x_coap+timing_coap_nb_res.intercept
y_coap_ltem_t = timing_coap_ltem_res.slope*x_coap+timing_coap_ltem_res.intercept

ax1.plot(x_coap,y_coap_nb_t-20.48, "g", label="NB-IoT regression line")
ax1.scatter(bytes_coap, np.array(timing_coap_nb[0])-20.48, color = "g", s=5, alpha=0.3, label="NB-IoT data")
ax1.scatter(bytes_coap, np.array(timing_coap_nb[1])-20.48, color = "g", s=5, alpha=0.3)   
ax1.scatter(bytes_coap, np.array(timing_coap_nb[2])-20.48, color = "g", s=5, alpha=0.3) 

ax1.plot(x_coap,y_coap_ltem_t-10.24, "r", label="LTE-M regression line") #subtracting the RRC Connected time of LTE-M. This is not consistent though 
ax1.scatter(bytes_coap, np.array(timing_coap_ltem[0])-10.24, color = "r", s=5, alpha=0.3, label="LTE-M data")
ax1.scatter(bytes_coap, np.array(timing_coap_ltem[1])-10.24, color = "r", s=5, alpha=0.3)   
ax1.scatter(bytes_coap, np.array(timing_coap_ltem[2])-10.24, color = "r", s=5, alpha=0.3)  

## MQTT
ax2.set_title("MQTT")
ax2.set_xlabel("Payload size [Bytes]")

y_mqtt_nb_t   =   timing_mqtt_nb_res.slope  *x_mqtt+timing_mqtt_nb_res.intercept
y_mqtt_ltem_t = timing_mqtt_ltem_res.slope*x_mqtt+timing_mqtt_ltem_res.intercept

ax2.plot(x_mqtt,y_mqtt_nb_t-20.48, "g", label="NB-IoT regression line")
ax2.scatter(bytes_mqtt, np.array(timing_mqtt_nb[0])-20.48, color = "g", s=5, alpha=0.3, label="NB-IoT data")
ax2.scatter(bytes_mqtt, np.array(timing_mqtt_nb[1])-20.48, color = "g", s=5, alpha=0.3)   
ax2.scatter(bytes_mqtt, np.array(timing_mqtt_nb[2])-20.48, color = "g", s=5, alpha=0.3) 

ax2.plot(x_mqtt,y_mqtt_ltem_t-10.24, "r", label="LTE-M regression line") #subtracting the RRC Connected time of LTE-M. This is not consistent though 
ax2.scatter(bytes_mqtt, np.array(timing_mqtt_ltem[0])-10.24, color = "r", s=5, alpha=0.3, label="LTE-M data")
ax2.scatter(bytes_mqtt, np.array(timing_mqtt_ltem[1])-10.24, color = "r", s=5, alpha=0.3)   
#ax2.scatter(bytes_mqtt, np.array(timing_mqtt_ltem[2])*delta_t-10.24, color = "r", s=5, alpha=0.3)  

ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.savefig("plots/time_reg_lines.pdf", bbox_inches='tight')


intersect = (energy_mqtt_nb_res.intercept-energy_mqtt_ltem_res.intercept)/(energy_mqtt_ltem_res.slope-energy_mqtt_nb_res.slope)
print("Energy consumption intersection at: " + str(intersect))

In [ ]:
### The following cells plots the calculated transmission times and energy ###

In [ ]:

plt.title("NB-IoT message energy")
plt.ylabel("Message energy [uWh]")
plt.xlabel("Payload size [Bytes]")

xMax = 4096
yMin = 30
yMax = 110

plt.scatter(bytes_coap, coap_nb_sums[0], color = "g", s=5)
plt.scatter(bytes_coap, coap_nb_sums[1], color = "g", s=5)   
plt.scatter(bytes_coap, coap_nb_sums[2], color = "g", s=5, label = "CoAP") 


plt.scatter(bytes_mqtt, mqtt_nb_sums[0], color = "r", s=5)
plt.scatter(bytes_mqtt, mqtt_nb_sums[1], color = "r", s=5)   
plt.scatter(bytes_mqtt, mqtt_nb_sums[2], color = "r", s=5, label = "MQTT") 

nb_diffs = []
for i in range(len(timing_coap_nb)):
    curr_coap = np.array(coap_nb_sums[i])
    curr_mqtt = np.array(mqtt_nb_sums[i][:len(curr_coap)])

    nb_diffs.append(100*(1-curr_coap/curr_mqtt))

print(np.mean(nb_diffs))
print(np.std(nb_diffs))

plt.legend()

plt.savefig("../dt_thesis/plots/energy_nbiot.pdf", bbox_inches='tight')

In [ ]:
plt.title("LTE-M message energy")
plt.ylabel("Message energy [uWh]")
plt.xlabel("Payload size [Bytes]")

plt.scatter(bytes_coap, coap_ltem_sums[0], color = "g", s=5)
plt.scatter(bytes_coap, coap_ltem_sums[1], color = "g", s=5)   
plt.scatter(bytes_coap, coap_ltem_sums[2], color = "g", s=5, label = "CoAP") 

plt.scatter(bytes_mqtt, mqtt_ltem_sums[0], color = "r", s=5)
plt.scatter(bytes_mqtt, mqtt_ltem_sums[1], color = "r", s=5, label = "MQTT")   
#plt.scatter(bytes_mqtt, mqtt_ltem_sums[2], color = "g", s=5) 

plt.savefig("../dt_thesis/plots/energy_ltem.pdf", bbox_inches='tight')


In [ ]:
plt.title("NB-IoT transmission time")
plt.ylabel("Transmission time [s]")
plt.xlabel("Payload size [Bytes]")

plt.scatter(bytes_coap, np.array(timing_coap_nb[0])-20.48, color = "g", s=5, label="CoAP")
plt.scatter(bytes_coap, np.array(timing_coap_nb[1])-20.48, color = "g", s=5)   
plt.scatter(bytes_coap, np.array(timing_coap_nb[2])-20.48, color = "g", s=5) 

plt.scatter(bytes_mqtt, np.array(timing_mqtt_nb[0])-20.48, color = "r", s=5, label="MQTT")
plt.scatter(bytes_mqtt, np.array(timing_mqtt_nb[1])-20.48, color = "r", s=5)   
plt.scatter(bytes_mqtt, np.array(timing_mqtt_nb[2])-20.48, color = "r", s=5) 


nb_diffs = []
for i in range(len(timing_coap_nb)):
    curr_coap = np.array(timing_coap_nb[i])
    curr_mqtt = np.array(timing_mqtt_nb[i][:len(curr_coap)])

    nb_diffs.append(100*(1-curr_coap/curr_mqtt))

print(np.mean(nb_diffs))
print(np.std(nb_diffs))
plt.savefig("../dt_thesis/plots/time_nbiot.pdf", bbox_inches='tight')

In [ ]:
plt.title("LTE-M transmission time")
plt.ylabel("Transmission time [s]")
plt.xlabel("Payload size [Bytes]")

plt.scatter(bytes_coap, np.array(timing_coap_ltem[0])-10.24, color = "g", s=5, label="CoAP")
plt.scatter(bytes_coap, np.array(timing_coap_ltem[1])-10.24, color = "g", s=5)   
plt.scatter(bytes_coap, np.array(timing_coap_ltem[2])-10.24, color = "g", s=5)  

plt.scatter(bytes_mqtt, np.array(timing_mqtt_ltem[0])-10.24, color = "r", s=5, label="MQTT")
plt.scatter(bytes_mqtt, np.array(timing_mqtt_ltem[1])-10.24, color = "r", s=5)   


plt.savefig("../dt_thesis/plots/time_ltem.pdf", bbox_inches='tight')

In [ ]:
### Following this cell is calculation and plotting of residuals ###
# This was not used directly in the thesis #

In [ ]:
residuals_coap_nb   = get_residuals(coap_nb_sums,   energy_coap_nb_res,     bytes_coap)
residuals_coap_ltem = get_residuals(coap_ltem_sums, energy_coap_ltem_res,   bytes_coap)
residuals_mqtt_nb   = get_residuals(mqtt_nb_sums,   energy_mqtt_nb_res,     bytes_mqtt)
residuals_mqtt_ltem = get_residuals(mqtt_ltem_sums, energy_mqtt_ltem_res,   bytes_mqtt)

In [ ]:
fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2)

ax1.set_title("Coap NB-IoT residuals")
ax2.set_title("Coap LTE-M residuals")
ax3.set_title("MQTT NB-IoT residuals")
ax4.set_title("MQTT LTE-M residuals")


ax1.scatter(residuals_coap_nb[0],   residuals_coap_nb[1])
ax2.scatter(residuals_coap_ltem[0], residuals_coap_ltem[1])
ax3.scatter(residuals_mqtt_nb[0],   residuals_mqtt_nb[1])
ax4.scatter(residuals_mqtt_ltem[0], residuals_mqtt_ltem[1])


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)

fig.suptitle("Message energy regression residuals")

ax1.set(ylabel="Energy [uWh]")
y_min = -70
y_max = 110

ax1.set_ylim([y_min,y_max])
ax2.set_ylim([y_min,y_max])

ax1.set_title("CoAP")
ax1.set_xticks([1,2])
ax1.set_xticklabels(['NB-IoT', "LTE-M"])
ax2.set_title("MQTT")
ax2.set_xticks([1,2])
ax2.set_xticklabels(['NB-IoT', "LTE-M"])

ax1.violinplot(residuals_coap_nb[1], positions = [1])
ax1.violinplot(residuals_coap_ltem[1], positions = [2])

ax2.violinplot(residuals_mqtt_nb[1], positions = [1])
ax2.violinplot(residuals_mqtt_ltem[1], positions = [2])

plt.savefig("plots/energy_reg_res.pdf", bbox_inches='tight')


In [ ]:
residuals_time_coap_nb   = get_residuals(timing_coap_nb,   timing_coap_nb_res,     bytes_coap)
residuals_time_coap_ltem = get_residuals(timing_coap_ltem, timing_coap_ltem_res,   bytes_coap)
residuals_time_mqtt_nb   = get_residuals(timing_mqtt_nb,   timing_mqtt_nb_res,     bytes_mqtt)
residuals_time_mqtt_ltem = get_residuals(timing_mqtt_ltem, timing_mqtt_ltem_res,   bytes_mqtt)

In [ ]:
fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(2,2)

ax1.set_title("Coap NB-IoT time spent - residuals")
ax2.set_title("Coap LTE-M time spent - residuals")
ax3.set_title("MQTT NB-IoT time spent - residuals")
ax4.set_title("MQTT LTE-M time spent - residuals")

ax1.scatter(residuals_time_coap_nb[0]  , residuals_time_coap_nb[1])
ax2.scatter(residuals_time_coap_ltem[0], residuals_time_coap_ltem[1])
ax3.scatter(residuals_time_mqtt_nb[0]  , residuals_time_mqtt_nb[1]    )
ax4.scatter(residuals_time_mqtt_ltem[0], residuals_time_mqtt_ltem[1])


In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)

fig.suptitle("Transmit time regression residuals")

ax1.set(ylabel="Time [s]")
y_min = -5
y_max = 11

ax1.set_ylim([y_min,y_max])
ax2.set_ylim([y_min,y_max])

ax1.set_title("CoAP")
ax1.set_xticks([1,2])
ax1.set_xticklabels(['NB-IoT', "LTE-M"])
ax2.set_title("MQTT")
ax2.set_xticks([1,2])
ax2.set_xticklabels(['NB-IoT', "LTE-M"])

ax1.violinplot(residuals_time_coap_nb[1]  , positions = [1])
ax1.violinplot(residuals_time_coap_ltem[1], positions = [2])

ax2.violinplot(residuals_time_mqtt_nb[1]  , positions = [1])
ax2.violinplot(residuals_time_mqtt_ltem[1], positions = [2])

plt.savefig("plots/time_reg_res.pdf", bbox_inches='tight')

